In [ ]:
from sklearn.ensemble import VotingClassifier 
import pickle
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score,recall_score,confusion_matrix
import csv

In [ ]:
ransom_test = pd.read_csv('/content/sample_data/files/ransom_dataset_test25samples.csv', nrows=25)
benign_test = pd.read_csv('/content/sample_data/files/benign_dataset_test25samples.csv', nrows=25)

In [ ]:
ransom_test1 = ransom_test[ransom_test.columns[0:225000]]
benign_test1 = benign_test[benign_test.columns[0:225000]]

ransom_test2 = ransom_test[ransom_test.columns[0:175000]]
benign_test2 = benign_test[benign_test.columns[0:175000]]

ransom_test3 = ransom_test[ransom_test.columns[0:125000]]
benign_test3 = benign_test[benign_test.columns[0:125000]]

ransom_test4 = ransom_test[ransom_test.columns[0:75000]]
benign_test4 = benign_test[benign_test.columns[0:75000]]

In [ ]:
ransom_test1['class']=1
benign_test1['class']=0

ransom_test2['class']=1
benign_test2['class']=0

ransom_test3['class']=1
benign_test3['class']=0

ransom_test4['class']=1
benign_test4['class']=0

In [ ]:
test_df1 = pd.concat([ransom_test1,benign_test1],ignore_index=True)
test_df1 = test_df1.fillna(0)

test_df2 = pd.concat([ransom_test2,benign_test2],ignore_index=True)
test_df2 = test_df2.fillna(0)

test_df3 = pd.concat([ransom_test3,benign_test3],ignore_index=True)
test_df3 = test_df3.fillna(0)

test_df4 = pd.concat([ransom_test4,benign_test4],ignore_index=True)
test_df4 = test_df4.fillna(0)

In [ ]:
x_test1 = test_df1.drop('class',axis=1)


x_test2 = test_df2.drop('class',axis=1)


x_test3 = test_df3.drop('class',axis=1)


x_test4 = test_df4.drop('class',axis=1)


X_test=[x_test1,x_test2,x_test3,x_test4]
Y_test=test_df4['class']



In [ ]:
a=['saved_models/dt1.sav','saved_models/dt2.sav','saved_models/dt3.sav','saved_models/dt4.sav','saved_models/gb1.sav','saved_models/gb2.sav','saved_models/gb3.sav','saved_models/gb4.sav','saved_models/gnb1.sav','saved_models/gnb2.sav','saved_models/gnb3.sav','saved_models/gnb4.sav','saved_models/knn1.sav','saved_models/knn2.sav','saved_models/knn3.sav','saved_models/knn4.sav','saved_models/log1.sav','saved_models/log2.sav','saved_models/log3.sav','saved_models/log4.sav','saved_models/rf1.sav','saved_models/rf2.sav','saved_models/rf3.sav','saved_models/rf4.sav','saved_models/svm1.sav','saved_models/svm2.sav','saved_models/svm3.sav','saved_models/svm4.sav']
l='/content/sample_data/'
filenames=[]
for i in a:
  filenames.append(l+i)

In [ ]:
from itertools import chain, combinations

def all_sublists(l):
    return chain(*(combinations(l, i) for i in range(len(l) + 1)))

def filtered_sublists(input_list, length):
    return (
        l for l in all_sublists(input_list)
        if len(l) == length 
    )
c=filtered_sublists(filenames,4)
combinations=list(c)

In [ ]:
len(combinations) #combinations of four models

In [ ]:
def predict(estimators,X_test):
  predictions=np.asarray([est.predict(X) for est, X in zip(estimators, X_test)]).T
  maj = np.apply_along_axis(lambda x: np.argmax(np.bincount(x,weights=None)),axis=1, arr=predictions)
  return maj

In [ ]:

with open('/content/sample_data/files/Voting4ModelsV2.csv','w') as f:
  w=csv.writer(f)
  w.writerow(['Models','Accuracy','Recall','Confusion Matrix'])
  for c in range(9085,len(combinations)):
    models=[pickle.load(open(i, 'rb')) for i in combinations[c]]
    names=list(s.split('/')[4].split('.')[0] for s in combinations[c] )
    datalist=list(int(x[-1])-1 for x in names)
    testdata=[X_test[i] for i in datalist]
    pred=predict(models,testdata)
    acc=accuracy_score(Y_test,pred)
    recall=recall_score(Y_test,pred)
    conf=confusion_matrix(Y_test,pred)    
    w.writerow([names.__str__()[1:-1].replace("'",""),acc,recall,conf.reshape(1,4).tolist().__str__()[2:-2]])
    

  

  
